In [18]:
import requests

class StorageNode:
    def __init__(self, name=None, host=None): 
        self.name = name
        self.host = host
        
        #Gets the file from instances of storage node
        def fetch_file(self, path):
            return requests.get(f"https://{self.host}:1231/{path}").text
        
        #Puts the file on the instances of storage node
        def put_file(self, path):
            with open(path, 'r') as fp:
                content = fp.read()
                return requests.post(f"https://{self.host}:1231/{path}", body=content).text

In [19]:
#Storage node holding instances of actual storage node objects

#Creating an array to keep track of of storage node(servers) and it's respective IP address
storage_nodes = [
    StorageNode(name="A", host="192.168.20.42"),
    StorageNode(name="B", host="192.168.20.52"),
    StorageNode(name="C", host="192.168.20.62"),
    StorageNode(name="D", host="192.168.20.72"),
    ]

#Hash the storage node. 
"""
Working: sums the bytes present in the 'key' and then take the mod of 
it with no. of storage node. Thus generating output in the range of[0, len-1]
"""
def hash_fn(key):
    return sum(bytearray(key.encode('utf-8'))) % len(storage_nodes)

In [20]:
# Maps the file on the ring to the nearest storage node
def upload(path):
    index = hash_fn(path) #To get the index of the storage node that would hold the file
    node = storage_nodes[index] #We get storage node instance
     
    return node.put_file(path) #We put the file on the node and return it

# Fetches the file from the instance of storage node
def fetch(path):
    index = hash_fn(path) #To get the index of the storage node which is holding the file
    node = storage_nodes[index] #We get storage node instance
    
    return node.fetch_file(path) #We fetch the file from the node and return

In [ ]:
#Files to be stored on the instance of storage node
files = ["f1.txt", "f2.txt", "f3.txt", "f4.txt", "f5.txt", "f6.txt", "f7.txt"]

#Mapping which file will be stored on which instance
for file in files:
    print(f"FILE: {file} ---> NODE: {storage_nodes[hash_fn(file)].name} on HOST: {storage_nodes[hash_fn(file)].host}")

In [22]:
import hashlib
from bisect import bisect, bisect_left, bisect_right
from plot import plot_hashring

In [23]:
"""
Creates an integer equivalent of SHA256 hash and takes modulo with the
total no of slots in hash space
"""
def hash_fn(key: str, total_slots: int) -> int:
    hsh = hashlib.sha256() 
    hsh.update(bytes(key.encode('utf-8'))) #Converting data into bytes and passing it to hash function
    
    #Converting the HEX digest into equivalent integer value
    return int(hsh.hexdigest(),16) % total_slots

In [24]:
 #Represents an array based implementation of consistent hashing
class ConsistentHash: 
   
    def __init__(self):
        self._keys = [] #Indices taken up in the ring
        self.nodes = [] #Nodes present in the ring
        self.total_slots = 100 #Total no. of slots
    
    #Adds new node in the system and return the hash space after placing it on the ring
    def add_node(self, node:StorageNode) -> int:
        #If no. of instances equals to total slots
        if len(self._keys) == self.total_slots: 
            raise Exception("HASH SPACE IS FULL")
            
        #Gets the key using hash_fn by hashing the host(IP) of the storage node
        key = hash_fn(node.host, self.total_slots) 
        
        #Finds the index where the key should be inserted in the key array
        #This will be the index where the storage node will be added in the node
        index = bisect(self._keys, key)
        
        #If Node already present for the same key
        if index > 0 and self._keys[index-1] == key:
            raise Exception("COLLISION OCCURED")
        
        #Insert the node and the key at the same index location.
        #this will keep node and keys sorted
        self.nodes.insert(index, node)
        self._keys.insert(index, key)
    
    #Remove node and returns the key from the hash space on which the node was placed
    def remove_node(self, node: StorageNode) -> int:
        #If hash space was empty
        if len(self._keys) == 0: 
            raise Exception("HASH SPACE IS EMPTY")
        #Gets the key using hash_fn by hashing the host(IP) of the storage node
        key = hash_fn(node.host, self.total_slots)
        
        #Finds the index where the key would reside on
        index = bisect_left(self._keys, key)
        
        #If key does not exist in the array
        if index >= len(self._keys) or self._keys[index] != key:
            raise Exception("NODE DOES NOT EXIST")
        
        #We pop the key and the node from the index
        self._keys.pop(index)
        self.nodes.pop(index)
        
        return key
    
    #Given a file, the function returns the node it is associated with
    def assign(self, item:str) -> str:
        #Gets the key using hash_fn by hashing the host(IP) of the storage node
        key = hash_fn(item, self.total_slots)
        
        #Finds the first index right of the key
        #If bisect_right returns index which is out of bound the we circle back to the first in the array
        index = bisect_right(self._keys, key) % len(self._keys)
        
        #Returns the node present at the index
        return self.nodes[index]
    
    #Plots the the storage and file 
    def plot(self, item:str = None, node: StorageNode = None) -> None:
        plot_hashring(
            self.total_slots,
            self._keys,
            self.nodes,
            item_key = hash_fn(item, self.total_slots) if item else None,
            node_key = hash_fn(node.host, self.total_slots) if node else None,
        )

In [25]:
ch = ConsistentHash()

In [ ]:
for node in storage_nodes:
    ch.add_node(node) #Adds node in the consistent hash
ch.plot() #Plotting the nodes(storage node)

In [ ]:
for file in files:
        print(f"FILE: {file}(in green) --->  NODE: {ch.assign(file).name}(in red)")
        ch.plot(file, ch.assign(file))